In [5]:
###################[0926_0084_200frame]###################

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
%matplotlib inline

def Evaluate(pre_disp_path, gt_depth_path,name):
    min_depth = 1e-3
    max_depth = 80
    
    predicted_disp_path = pre_disp_path
    gt_path = gt_depth_path
    
    rms     = np.zeros(num_samples, np.float32)
    log_rms = np.zeros(num_samples, np.float32)
    abs_rel = np.zeros(num_samples, np.float32)
    sq_rel  = np.zeros(num_samples, np.float32)
    d1_all  = np.zeros(num_samples, np.float32)
    a1      = np.zeros(num_samples, np.float32)
    a2      = np.zeros(num_samples, np.float32)
    a3      = np.zeros(num_samples, np.float32)

    width_to_focal = dict()
    width_to_focal[1242] = 721.5377
    width_to_focal[1241] = 718.856
    width_to_focal[1224] = 707.0493
    width_to_focal[1238] = 718.3351

    def compute_errors(gt, pred):
        thresh = np.maximum((gt / pred), (pred / gt))
        a1 = (thresh < 1.25   ).mean()
        a2 = (thresh < 1.25 ** 2).mean()
        a3 = (thresh < 1.25 ** 3).mean()

        rmse = (gt - pred) ** 2
        rmse = np.sqrt(rmse.mean())
        rmse_log = (np.log(gt) - np.log(pred)) ** 2
        rmse_log = np.sqrt(rmse_log.mean())
        abs_rel = np.mean(np.abs(gt - pred) / gt)
        sq_rel = np.mean(((gt - pred)**2) / gt)

        return abs_rel, sq_rel, rmse, rmse_log, a1, a2, a3

    def load_gt_depth_kitti(path, num_samples):
        gt_depths = []
        for i in range (num_samples):
            depth = cv2.imread(path + str(i).zfill(4) + ".png", -1)
            depth = depth.astype(np.float32) / 256
            #print("aaaaaaaaaaadepth",depth.max())
            gt_depths.append(depth)
        return gt_depths

    def convert_disps_to_depths_kitti(gt_depth, pred_disparities):
        pred_depths = []
        pred_disparities_resized = []

        for i in range(len(gt_depth)):
            height, width = gt_depth[i].shape
            #print('gt_depth[i].shape',gt_depth[i].shape)
            pred_disp = pred_disparities[i]
            #print(pred_disp.max())
            pred_disp = width * cv2.resize(pred_disp, (width, height), interpolation=cv2.INTER_LINEAR)
            #print('pred_disp',pred_disp.max())
            pred_disparities_resized.append(pred_disp) 
            np.set_printoptions(threshold=np.inf)

            pred_depth = width_to_focal[width] * 0.54 / pred_disp
            #print("depth",pred_depth.max())
            pred_depths.append(pred_depth)

        return pred_depths, pred_disparities_resized

    gt_depths = load_gt_depth_kitti(gt_path, num_samples)
    
    pred_disparities = np.load(predicted_disp_path)
    pred_depths, pred_disparities_resized = convert_disps_to_depths_kitti(gt_depths, pred_disparities)

    for i in range(num_samples):
        gt_depth = gt_depths[i]
        pred_depth = pred_depths[i]
        pred_depth[pred_depth < min_depth] = min_depth
        pred_depth[pred_depth > max_depth] = max_depth
        mask = gt_depth > 0
        
        abs_rel[i], sq_rel[i], rms[i], log_rms[i], a1[i], a2[i], a3[i] = compute_errors(gt_depth[mask], 
                                                                                        pred_depth[mask])
        #print("{:10.4f}, {:10.4f}, {:10.3f}, {:10.3f}, {:10.3f}, {:10.3f}, {:10.3f}".format
          #(abs_rel[i], sq_rel[i], rms[i], log_rms[i], a1[i], a2[i], a3[i]))
#     plt.title('a1')       
#     x = a1
#     y = list(range(0, num_samples))
#     plt.scatter(y, x)
#     plt.savefig('/home/nung/Desktop/evaluate/200/'+name+'_a1.png')
#     plt.show()
    
#     plt.title('abs_rel')
#     x1 = abs_rel
#     y1 = list(range(0, num_samples))
#     plt.scatter(y1, x1)
#     plt.savefig('/home/nung/Desktop/evaluate/200/'+name+'_abs.png')  
#     plt.show()
      
    print("{:>10}, {:>10}, {:>10}, {:>10}, {:>10}, {:>10}, {:>10}".format
          ('abs_rel', 'sq_rel', 'rms', 'log_rms', 'a1', 'a2', 'a3'))
    print("{:10.4f}, {:10.4f}, {:10.4f}, {:10.4f}, {:10.5f}, {:10.5f}, {:10.5f}".format
          (abs_rel.mean(), sq_rel.mean(), rms.mean(), log_rms.mean(), a1.mean(), a2.mean(), a3.mean()))
    
    outfile="/home/nung/Desktop/evaluate/evaluate_new.txt"
    file=open(outfile, "a")
    file.write('200frame'+' '+name+'\n'+str(abs_rel.mean())+' '+str(sq_rel.mean())+' '+str(rms.mean())+' '+str(log_rms.mean())+' '+
               str(a1.mean())+' '+str(a2.mean())+' '+str(a3.mean())+'\n')

num_samples = 200
gt_path = '/home/nung/data/kitti/testtttt/depth/20110926_0084/'    


def Go_origin():
    name='origin'
    print('origin/'+str(num_samples)+' testing')
    predicted_disp_path = '/home/nung/aaaMONO/original/test/200/disparities_pp.npy'
    Evaluate(predicted_disp_path,gt_path, name)

def Go_origin_resnet():
    name='origin_resnet'
    print('origin_resnet/'+str(num_samples)+' testing')
    predicted_disp_path = '/home/nung/aaaMONO/original/test_resnet/200/disparities_pp.npy'
    Evaluate(predicted_disp_path,gt_path, name)    
    
def Go_STF_22():
    name='STF_22'
    print('STF_22fg/'+str(num_samples)+' testing')
    predicted_disp_path = '/home/nung/aaaMONO/forgate_gate/STFB-22/c200/disparities_pp.npy'
    Evaluate(predicted_disp_path,gt_path, name)    

def Go_STF_55():
    name='STF_55'
    print('STF_55fg/'+str(num_samples)+' testing')
    predicted_disp_path = '/home/nung/aaaMONO/forgate_gate/STFB-55/c200/disparities_pp.npy'
    Evaluate(predicted_disp_path,gt_path, name)    
    
def Go_STF_35():
    name='STF_35'
    print('STF_35fg/'+str(num_samples)+' testing')
    predicted_disp_path = '/home/nung/aaaMONO/forgate_gate/STFB-35/c200/disparities_pp.npy'
    Evaluate(predicted_disp_path,gt_path, name)
    
def Go_BTF():
    name='BTF'
    print('BTFfg/'+str(num_samples)+' testing')
    predicted_disp_path = '/home/nung/aaaMONO/forgate_gate/BTFB/c200/disparities_pp.npy'
    Evaluate(predicted_disp_path,gt_path, name)    
    
def Go_FG():
    name='FG35'
    print('FG35/'+str(num_samples)+' testing')
    predicted_disp_path = '/home/nung/aaaMONO/forgate_gate/200/disparities_pp.npy'
    Evaluate(predicted_disp_path,gt_path, name)
    

def Go_origin_ckvgg():
    name='origin_ckvgg'
    print('origin_ckvgg/'+str(num_samples)+' testing')
    predicted_disp_path = '/home/nung/aaaMONO/original/test_ck/200/disparities_pp.npy'
    Evaluate(predicted_disp_path,gt_path, name)    
#Go_origin()
# Go_origin_resnet()
# Go_STF_22()
# Go_STF_55()
# Go_STF_35()
Go_BTF()
#Go_FG()
#Go_origin_ckvgg()

BTFfg/200 testing
   abs_rel,     sq_rel,        rms,    log_rms,         a1,         a2,         a3
    0.0390,     0.1564,     2.3505,     0.0728,    0.97785,    0.99638,    0.99918


In [6]:
###################[1003_0027_1095frame]###################

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
%matplotlib inline
def Evaluate(pre_disp_path, gt_depth_path, name):
    min_depth = 1e-3
    max_depth = 80
    
    predicted_disp_path = pre_disp_path
    gt_path = gt_depth_path
    
    rms     = np.zeros(num_samples, np.float32)
    log_rms = np.zeros(num_samples, np.float32)
    abs_rel = np.zeros(num_samples, np.float32)
    sq_rel  = np.zeros(num_samples, np.float32)
    d1_all  = np.zeros(num_samples, np.float32)
    a1      = np.zeros(num_samples, np.float32)
    a2      = np.zeros(num_samples, np.float32)
    a3      = np.zeros(num_samples, np.float32)

    width_to_focal = dict()
    width_to_focal[1242] = 721.5377
    width_to_focal[1241] = 718.856
    width_to_focal[1226] = 707.0493
    width_to_focal[1238] = 718.3351

    def compute_errors(gt, pred):
        thresh = np.maximum((gt / pred), (pred / gt))
        a1 = (thresh < 1.25   ).mean()
        a2 = (thresh < 1.25 ** 2).mean()
        a3 = (thresh < 1.25 ** 3).mean()

        rmse = (gt - pred) ** 2
        rmse = np.sqrt(rmse.mean())
        rmse_log = (np.log(gt) - np.log(pred)) ** 2
        rmse_log = np.sqrt(rmse_log.mean())
        abs_rel = np.mean(np.abs(gt - pred) / gt)
        sq_rel = np.mean(((gt - pred)**2) / gt)

        return abs_rel, sq_rel, rmse, rmse_log, a1, a2, a3

    def load_gt_depth_kitti(path, num_samples):
        gt_depths = []
        for i in range (num_samples):
            depth = cv2.imread(path + str(i).zfill(4) + ".png", -1)
            depth = depth.astype(np.float32) / 256
            gt_depths.append(depth)
        return gt_depths

    def convert_disps_to_depths_kitti(gt_depth, pred_disparities):
        pred_depths = []
        pred_disparities_resized = []

        for i in range(len(gt_depth)):
            height, width = gt_depth[i].shape
            #print('gt_depth[i].shape',gt_depth[i].shape)
            pred_disp = pred_disparities[i]
            pred_disp = width * cv2.resize(pred_disp, (width, height), interpolation=cv2.INTER_LINEAR)
            #print('pred_disp.shape',pred_disp.shape)
            pred_disparities_resized.append(pred_disp) 
            np.set_printoptions(threshold=np.inf)
            pred_depth = width_to_focal[width] * 0.54 / pred_disp
            pred_depths.append(pred_depth)

        return pred_depths, pred_disparities_resized

    gt_depths = load_gt_depth_kitti(gt_path, num_samples)

    pred_disparities = np.load(predicted_disp_path)
    pred_depths, pred_disparities_resized = convert_disps_to_depths_kitti(gt_depths, pred_disparities)

    for i in range(num_samples):
        gt_depth = gt_depths[i]
        pred_depth = pred_depths[i]
        pred_depth[pred_depth < min_depth] = min_depth
        pred_depth[pred_depth > max_depth] = max_depth
        mask = gt_depth > 0
        abs_rel[i], sq_rel[i], rms[i], log_rms[i], a1[i], a2[i], a3[i] = compute_errors(gt_depth[mask], 
                                                                                        pred_depth[mask])

    print("{:>10}, {:>10}, {:>10}, {:>10}, {:>10}, {:>10}, {:>10}".format
          ('abs_rel', 'sq_rel', 'rms', 'log_rms', 'a1', 'a2', 'a3'))
    print("{:10.4f}, {:10.4f}, {:10.4f}, {:10.4f}, {:10.5f}, {:10.5f}, {:10.5f}".format
          (abs_rel.mean(), sq_rel.mean(), rms.mean(), log_rms.mean(), a1.mean(), a2.mean(), a3.mean()))
    
    outfile="/home/nung/Desktop/evaluate/evaluate_new.txt"
    file=open(outfile, "a")
    file.write('1095frame'+' '+name+'\n'+str(abs_rel.mean())+' '+str(sq_rel.mean())+' '+str(rms.mean())+' '+str(log_rms.mean())+' '+
               str(a1.mean())+' '+str(a2.mean())+' '+str(a3.mean())+'\n')
num_samples = 1095
gt_path = '/home/nung/data/kitti/testtttt/depth/20110930_0027/'



def Go_origin():
    name='origin'
    print('origin/'+str(num_samples)+' testing')
    predicted_disp_path = '/home/nung/aaaMONO/original/test/1096/disparities_pp.npy'
    Evaluate(predicted_disp_path,gt_path, name)

def Go_origin_resnet():
    name='origin_resnet'
    print('origin_resnet/'+str(num_samples)+' testing')
    predicted_disp_path = '/home/nung/aaaMONO/original/test_resnet/1096/disparities_pp.npy'
    Evaluate(predicted_disp_path,gt_path, name)    
    
def Go_STF_22():
    name='STF_22'
    print('STF_22fg/'+str(num_samples)+' testing')
    predicted_disp_path = '/home/nung/aaaMONO/forgate_gate/STFB-22/c1096/disparities_pp.npy'
    Evaluate(predicted_disp_path,gt_path, name)    

def Go_STF_55():
    name='STF_55'
    print('STF_55fg/'+str(num_samples)+' testing')
    predicted_disp_path = '/home/nung/aaaMONO/forgate_gate/STFB-55/c1096/disparities_pp.npy'
    Evaluate(predicted_disp_path,gt_path, name)    
    
def Go_STF_35():
    name='STF_35'
    print('STF_35fg/'+str(num_samples)+' testing')
    predicted_disp_path = '/home/nung/aaaMONO/forgate_gate/STFB-35/c1096/disparities_pp.npy'
    Evaluate(predicted_disp_path,gt_path, name)
    
def Go_FG():
    name='FG35'
    print('FG35/'+str(num_samples)+' testing')
    predicted_disp_path = '/home/nung/aaaMONO/forgate_gate/1095/disparities_pp.npy'
    Evaluate(predicted_disp_path,gt_path, name)
    
def Go_BTF():
    name='BTF'
    print('BTFfg/'+str(num_samples)+' testing')
    predicted_disp_path = '/home/nung/aaaMONO/forgate_gate/BTFB/c1096/disparities_pp.npy'
    Evaluate(predicted_disp_path,gt_path, name)    
def Go_origin_ckvgg():
    name='origin_ckvgg'
    print('origin_ckvgg/'+str(num_samples)+' testing')
    predicted_disp_path = '/home/nung/aaaMONO/original/test_ck/1095/disparities_pp.npy'
    Evaluate(predicted_disp_path,gt_path, name)   
# Go_origin()
# Go_origin_resnet()
# Go_STF_22()
# Go_STF_55()
# Go_STF_35()
Go_BTF()
#Go_origin_ckvgg()
#Go_FG()

BTFfg/1095 testing
   abs_rel,     sq_rel,        rms,    log_rms,         a1,         a2,         a3
    0.0360,     0.1002,     1.7081,     0.0662,    0.98442,    0.99655,    0.99888
